In [1]:
import pandas as pd
pd.options.display.max_columns = 200

import numpy as np
import pickle

import matplotlib.pyplot as plt
%matplotlib inline

# Данные

Зайдите на https://www.drivendata.org/ и зарегистрируйтесь. Для сегодняшней домашки будем данные брать именно отсюда.

Нас интересует конкурс https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/23/ .
В нем представлены данные, собранные [Taarifa](taarifa.org) и [Танзанийским Министерством Воды и Ирригации](https://www.maji.go.tz/?q=en).

Постановка задачи следующая: 
На территории Танзании установлено множество водяных насосов, которые спасают местное население от жажды. В зависимости от того, кем и когда установлен насос, а также зная, как им распоряжаются, можно попытаться предположить, какие из них функционируют, какие нуждаются в ремонте и какие не работают вовсе. 

Этим мы и займемся, а заодно и прокачаемся в подборе гиперпараметров алгоритмов.

In [2]:
train_X, train_y = pd.read_csv( # путь к вашему файлу train.csv
    '../Kaggle/PumpItUp/train.csv'
), pd.read_csv( # путь к вашему файлу trainLabels.csv
    '../Kaggle/PumpItUp/trainLabels.csv'
)
df = pd.merge(train_X, train_y, how='left')

df_test = pd.read_csv( # путь к вашему файлу test.csv
    '../Kaggle/PumpItUp/test.csv'
)

In [4]:
# train_X.head()

# Предобработка

<div class="panel panel-warning">
    <div class="panel-heading">
        <h3 class="panel-title">Обратите внимание</h3> 
    </div>
</div>

Предобработка - опциональный блок, и у себя подготовить данные вы можете полностью по-своему.

__Единственное замечание:__ если решите подготавливать данные самостоятельно, замените странную строку `"other - mkulima/shinyanga"` на просто `"other"`, так как в тесте только `"other"`.
```python
df.loc[df.extraction_type == 'other - mkulima/shinyanga', 'extraction_type'] = 'other'
```

In [5]:
def reduce_factor_levels(df, column_name, limit=None, top=None, name=None):
    assert(limit is not None or top is not None), 'Specify limit ot top'
    if top is None:
        top = df[column_name].value_counts()[:limit].index
    if name is None:
        name = '%s_OTHER' % column_name
    df.loc[~df[column_name].isin(top), column_name] = name
    return top

In [6]:
top = reduce_factor_levels(df, 'funder', 10)
reduce_factor_levels(df_test, 'funder', top=top);

top = reduce_factor_levels(df, 'installer', 10)
reduce_factor_levels(df_test, 'installer', top=top);

In [7]:
drop = ['wpt_name', 'num_private', 'subvillage', 'region_code', 'district_code', 'lga', 'ward', 'recorded_by', 'scheme_name']

df.drop(drop, axis=1, inplace=True)
df_test.drop(drop, axis=1, inplace=True)

In [8]:
df.loc[df.scheme_management == 'None', 'scheme_management'] = ''
df.loc[df.scheme_management.isnull(), 'scheme_management'] = ''

df_test.loc[df_test.scheme_management.isnull(), 'scheme_management'] = ''

In [9]:
df['construction_date_known'] = (df.construction_year > 0).astype(np.int32)
df_test['construction_date_known'] = (df_test.construction_year > 0).astype(np.int32)

In [10]:
min_year = df[df.construction_year > 0].construction_year.min() // 10 - 1

df['construction_decade'] = df.construction_year // 10 - min_year
df_test['construction_decade'] = df_test.construction_year // 10 - min_year

df.loc[df.construction_decade < 0, 'construction_decade'] = 0
df_test.loc[df_test.construction_decade < 0, 'construction_decade'] = 0

In [11]:
top = reduce_factor_levels(df, 'construction_year', 20)
reduce_factor_levels(df_test, 'construction_year', top=top);

In [12]:
df.loc[df.extraction_type == 'other - mkulima/shinyanga', 'extraction_type'] = 'other'

In [13]:
heights = np.arange(-1, df.gps_height.max()+500, 500)
height_labels = list(range(len(heights)-1))

df['gps_height_rounded'] = pd.cut(df.gps_height, bins=heights, labels=height_labels)
df_test['gps_height_rounded'] = pd.cut(df_test.gps_height, bins=heights, labels=height_labels)

df.drop(['gps_height'], axis=1, inplace=True)
df_test.drop(['gps_height'], axis=1, inplace=True)

In [14]:
pops = np.arange(-1, df.population.max()+500, 500)
pops_labels = list(range(len(pops)-1))

df['pop_rounded'] = pd.cut(df.population, bins=pops, labels=pops_labels)
df_test['pop_rounded'] = pd.cut(df_test.population, bins=pops, labels=pops_labels)

df.drop(['population'], axis=1, inplace=True)
df_test.drop(['population'], axis=1, inplace=True)

In [15]:
df.drop(['date_recorded'], axis=1, inplace=True)
df_test.drop(['date_recorded'], axis=1, inplace=True)

In [16]:
df.public_meeting.fillna(True, inplace=True)
df_test.public_meeting.fillna(True, inplace=True)

In [17]:
df.permit.fillna(True, inplace=True)
df_test.permit.fillna(True, inplace=True)

In [18]:
df.gps_height_rounded.fillna(0, inplace=True)
df_test.gps_height_rounded.fillna(0, inplace=True)

# Визуализация

In [19]:
# Снова на ваше усмотрение
# Кстати, в этот раз данные с долготой-широтой, - отличная возможность поработать с геоданными!

# Модели

In [20]:
X, y, X_test = df.drop(['id', 'status_group'], axis=1), \
               df.status_group, \
               df_test.drop(['id'], axis=1)

In [21]:
X.head(1)

,amount_tsh,funder,installer,longitude,latitude,basin,region,public_meeting,scheme_management,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,construction_date_known,construction_decade,gps_height_rounded,pop_rounded
0,6000.0,funder_OTHER,installer_OTHER,34.938093,-9.856322,Lake Nyasa,Iringa,True,VWC,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,1,4,2,0


<div class="panel panel-warning">
    <div class="panel-heading">
        <h3 class="panel-title">Обратите внимание</h3> 
    </div>
</div>

Вот эта функция ниже - опять мои штуки-дрюки, и можно кодировать данные по-своему.

In [22]:
def prepare(X_train, X_test):
    from sklearn.preprocessing import StandardScaler
    from sklearn.feature_extraction import DictVectorizer
    
    objects = X_train.select_dtypes(include=['O']).columns.values
    numeric = X_train.select_dtypes(exclude=['O']).columns.values
    
    dv = DictVectorizer(sparse=False)
    data_encoded_tr = dv.fit_transform(X_train[objects].to_dict(orient='records'))
    data_encoded_ts = dv.transform(X_test[objects].to_dict(orient='records'))

    ss = StandardScaler()
    data_scaled_tr = ss.fit_transform(X_train[numeric])
    data_scaled_ts = ss.transform(X_test[numeric])
    
    train = np.hstack((data_encoded_tr, data_scaled_tr))
    test  = np.hstack((data_encoded_ts, data_scaled_ts))
    return train, test

In [23]:
x_train, x_test = prepare(X, X_test)

In [24]:
from sklearn.preprocessing import LabelEncoder

In [25]:
y_encoder = LabelEncoder()
y = y_encoder.fit_transform(y)

<div class="panel panel-info" style="margin: 50px 0 0 0">
    <div class="panel-heading">
        <h3 class="panel-title">Задание 1.</h3> 
    </div>
</div>

Возьмите тетрадку с сегодняшнего занятия и, руководствуясь советами по настройке, заделайте лучший GBM в мире! Не забудьте отправлять результаты на drivendata и хвастаться в чате о результатах.

In [ ]:
# Ваш код здесь

<div class="panel panel-info" style="margin: 50px 0 0 0">
    <div class="panel-heading">
        <h3 class="panel-title">Задание 2.</h3> 
    </div>
</div>

Выберите любой из сторонних фреймворков по своему усмотрению:
* XGBoost
* LightGBM
* H2O
* CatBoost

Установите, прокачайте его, побейте GBM от sklearn.

In [31]:
# Ваш код здесь

<div class="panel panel-info" style="margin: 50px 0 0 0">
    <div class="panel-heading">
        <h3 class="panel-title">Задание 3 (опционально).</h3> 
    </div>
</div>

Возьмите __hyperopt__ или его порт для классификаторов sklearn, называющийся __hyperopt-sklearn__. Установите его, попробуйте найти оптимальные гиперпараметры с помощью байесовской оптимизации. Помните, что это не silver bullet, и сценарий подбора оптимальный параметров все еще актуален. Но на этапе подбора параметров деревьев байесовская оптимизация может дать выигрыш во времени, уделав классический GridSearchCV.

In [ ]:
# Ваш код здесь